In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, callbacks
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler
import chardet
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import StandardScaler
from keras.models import Model, load_model
from keras.layers import Input, Dense

2025-10-24 06:19:03.944938: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-24 06:19:03.954458: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-24 06:19:04.286962: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-24 06:19:05.771964: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

In [3]:
import sys
ColabNotebook = 'google.colab' in str(get_ipython())

if ColabNotebook:
    # monta G-drive en entorno COLAB
    from google.colab import drive
    drive.mount('/content/drive/')

    # carpeta donde se encuentran archivos .py auxiliares
    FUENTES_DIR = '/content/drive/MyDrive/Colab Notebooks/FUENTES/'
    # carpeta donde se encuentran los datasets
    DATOS_DIR = '/content/drive/MyDrive/Colab Notebooks/DATOS/'
    LOCAL_DIR = './DATOS/'  # carpeta local en maquina virtual
else:
    # configuración para notebook con instalación LOCAL
    # carpeta donde se encuentran archivos .py auxiliares
    FUENTES_DIR = '../../FUENTES'
    DATOS_DIR = '../Datos/'  # carpeta LOCAL donde se encuentran los datasets
    LOCAL_DIR = DATOS_DIR


def openFile(nomArch, sep=None):
    file = DATOS_DIR + nomArch
    # -- detectando la codificación de caracteres usada ----
    with open(file, 'rb') as f:
        result = chardet.detect(f.read())
    # or readline if the file is large
    return pd.read_csv(file, encoding=result['encoding'], sep=sep, engine='python')


# agrega ruta de busqueda donde tenemos archivos .py
sys.path.append(FUENTES_DIR)

## Praparación de datos

En este bloque se realiza la preparación del conjunto de datos para entrenar y validar la red neuronal que colorea imágenes. Se parte de un conjunto de tuplas o pares de imágenes donde, donde la primera está en escala de grises (entrada del modelo) y la segunda corresponde a la versión en color (objetivo del modelo). Los pasos realizados en este proceso de generación de este conjunto de datos son:
* **Carga de imágenes**: Se leen las imágenes desde disco, asegurando que cada imagen en grises tenga su contraparte en color con el mismo nombre de archivo.
* **Preprocesamiento**: Las imágenes son redimensionadas a un tamaño fijo y normalizadas para que sus valores estén entre 0 y 1.
* **Estructuración como dataset TensorFlow**: Los datos se organizan en un objeto ***tf.data.Dataset***, que permite aplicar transformaciones eficientes como:
  * Mezclar aleatoriamente los ejemplos.
  * Aplicar procesamiento en paralelo.
  * Agrupar en lotes para entrenamiento por mini-lotes(minibatches).
  * Prefetch para optimizar el flujo de datos al modelo.
* **División entrenamiento-validación**:
  * **Entrenamiento (75%)**: utilizado por el modelo para aprender los patrones de coloreado.
  * **Validación (25%)**: utilizado para evaluar el rendimiento del modelo durante el entrenamiento y evitar sobreajuste.

In [12]:
import zipfile
import os

# Descomprimir en la misma carpeta
with zipfile.ZipFile(os.path.join(DATOS_DIR, 'landscape_color_gray.zip'), 'r') as zip_ref:
    zip_ref.extractall(os.path.join(DATOS_DIR, 'landscape_color_gray'))

data_dir = os.path.join(DATOS_DIR, 'landscape_color_gray')

In [13]:
# Configuración
TAM_IMG = 96   # Tamaño de la imagen que será procesada por la red
FACTOR_ENTRENA = 0.75  # Proporción del dataset para entrenamiento
TAM_LOTE = 64  # Tamaño de los lotes

ruta_color = '/landscape Images/color/'
ruta_grises = '/landscape Images/gray/'

# Función optimizada para cargar y procesar imágenes


@tf.function  # Esto convierte una función python en una versión tensorflow con el objetivo de mejorar el rendimiento
def load_and_process(gray_path, rgb_path, img_size=(TAM_IMG, TAM_IMG)):
    # Carga paralela de ambos archivos
    gray_img = tf.io.read_file(gray_path)
    rgb_img = tf.io.read_file(rgb_path)
    # Decodificación paralela
    gray_img = tf.image.decode_jpeg(gray_img, channels=1)
    rgb_img = tf.image.decode_jpeg(rgb_img, channels=3)
    # Redimensionar
    gray_img = tf.image.resize(gray_img, img_size)
    rgb_img = tf.image.resize(rgb_img, img_size)
    # Normalizar
    gray_img = tf.cast(gray_img, tf.float32) / 255.0
    rgb_img = tf.cast(rgb_img, tf.float32) / 255.0

    return gray_img, rgb_img

# Función optimizada para cargar el dataset


def cargar_dataset(gray_files, rgb_files, batch_size=32, shuffle_buffer=1000):
    # Crear dataset directamente desde las listas de archivos
    dataset = tf.data.Dataset.from_tensor_slices((gray_files, rgb_files))

    # Shuffle antes de mapear para mejor rendimiento
    if shuffle_buffer:
        dataset = dataset.shuffle(
            shuffle_buffer, reshuffle_each_iteration=False)
    # Mapeo paralelizado y cacheado
    dataset = dataset.map(
        load_and_process,
        num_parallel_calls=tf.data.AUTOTUNE
    )
    # Batch y prefetch
    dataset = dataset.batch(batch_size, drop_remainder=True)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

    return dataset


# Cargar listas de archivos una sola vez
gray_files = sorted([
    os.path.join(data_dir+ruta_grises, f) for f in os.listdir(data_dir+ruta_grises) if f.endswith('.jpg')
])

rgb_files =  sorted([
    os.path.join(data_dir+ruta_color, f) for f in os.listdir(data_dir+ruta_color) if f.endswith('.jpg')
])


# Verificar correspondencia
assert all(os.path.basename(g) == os.path.basename(r) for g, r in zip(gray_files, rgb_files)), \
    "¡Archivos no coincidentes!"

# Dividir dataset en train/val
cant_total = len(gray_files)
cant_entrena = int(FACTOR_ENTRENA * cant_total)

# Crear datasets directamente divididos
train_gray = gray_files[:cant_entrena]
train_rgb = rgb_files[:cant_entrena]

val_gray = gray_files[cant_entrena:]
val_rgb = rgb_files[cant_entrena:]

# Crear datasets finales
dataset_entrena = cargar_dataset(
    train_gray,
    train_rgb,
    batch_size=TAM_LOTE,
    shuffle_buffer=1000
)

dataset_valida = cargar_dataset(
    val_gray,
    val_rgb,
    batch_size=TAM_LOTE,
    shuffle_buffer=0  # No necesitamos shuffle para validación
)

# Opcional: Cachear los datasets si caben en memoria
dataset_entrena = dataset_entrena.cache()
dataset_valida = dataset_valida.cache()

## **Construccion de los modelos**

Construye el modelo utilizando capas convolucionales que reducen el tamaño de la imagen a través del stride e incrementan la cantidad de filtros hasta llegar al espacio de representación latente. Luego se realiza el proceso inverso agregando capas convolucionales que decrementan la cantidad de filtros y capas para ir recuperando el tamaño de la imagen al original.

In [ ]:
def entrenar_modelo(model, dataset_entrena, dataset_valida):
    es = callbacks.EarlyStopping(
        monitor='val_loss',  # Monitorea la pérdida de validación
        patience=15,         # Número de épocas sin mejora antes de detener
        restore_best_weights=True,  # Restaura los mejores pesos encontrados
    )

    history = model.fit(
        dataset_entrena,
        validation_data=dataset_valida,
        epochs=500,
        callbacks=[es],
        verbose=1
    )
    
    return history

Epoch 1/500
30/83 ━━━━━━━━━━━━━━━━━━━━ 7:23 8s/step - loss: 0.1737

KeyboardInterrupt: 

In [ ]:
histories = []

In [ ]:
def plot_comparasion(dataset_batch, predictions, num_images=5, start_index=0):
    """
    Muestra comparativas entre imágenes originales, escala de grises y predicciones del modelo

    Args:
        dataset_batch: Batch de datos (tupla de tensores (gray_images, color_images))
        predictions: Array de predicciones del modelo (num_images, H, W, 3)
        num_images: Número de imágenes a mostrar
        start_index: Índice inicial dentro del batch
    """
    gray_images = dataset_batch[0][start_index:start_index+num_images].numpy()
    color_images = dataset_batch[1][start_index:start_index+num_images].numpy()

    # Asegurar que las predicciones tienen el mismo número de imágenes
    predictions = predictions[start_index:start_index+num_images]

    fig, axes = plt.subplots(3, num_images, figsize=(16, 10))
    if num_images == 1:
        axes = axes.reshape(3, 1)  # Para manejar correctamente el caso de 1 imagen

    # Mostrar imágenes
    for i in range(num_images):
        # Escala de grises (eliminar dimensión del canal si es necesario)
        axes[0, i].imshow(gray_images[i].squeeze(), cmap='gray')
        axes[0, i].axis('off')

        # Original a color
        axes[1, i].imshow(color_images[i])
        axes[1, i].axis('off')

        # Predicción
        axes[2, i].imshow(predictions[i])
        axes[2, i].axis('off')

    # Etiquetas
    axes[0, 0].set_ylabel('Escala de Grises', fontsize=12)
    axes[1, 0].set_ylabel('Original Color', fontsize=12)
    axes[2, 0].set_ylabel('Predicción Modelo', fontsize=12)

    plt.tight_layout()
    plt.show()

### Upsampling2D

In [ ]:
from tensorflow.keras import layers, models, optimizers, activations

def crear_modelo(img_h, img_w, activ='relu'):
    inputs = layers.Input(shape=(img_h, img_w, 1))

    # --- Encoder ---
    x = layers.Conv2D(64, (3,3), activation=activ, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, (3,3), activation=activ, padding='same', strides=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(128, (3,3), activation=activ, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(128, (3,3), activation=activ, padding='same', strides=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(256, (3,3), activation=activ, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(256, (3,3), activation=activ, padding='same', strides=2)(x)
    x = layers.BatchNormalization()(x)

    # --- Bottleneck ---
    x = layers.Conv2D(512, (3,3), activation=activ, padding='same')(x)
    x = layers.BatchNormalization()(x)
    #x = layers.Conv2D(512, (3,3), activation=activ, padding='same')(x)
    #x = layers.BatchNormalization()(x)

    # --- Decoder ---
    x = layers.UpSampling2D(size=(2, 2))(x)
    x = layers.Conv2D(256, (3,3), activation=activ, padding='same')(x)
    x = layers.BatchNormalization()(x)

    x = layers.UpSampling2D(size=(2, 2))(x)
    x = layers.Conv2D(128, (3,3), activation=activ, padding='same')(x)
    x = layers.BatchNormalization()(x)

    x = layers.UpSampling2D(size=(2, 2))(x)
    x = layers.Conv2D(64, (3,3), activation=activ, padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Output
    outputs = layers.Conv2D(3, (3,3), activation='sigmoid', padding='same')(x)

    return models.Model(inputs=inputs, outputs=outputs)

    
# Crear y compilar el modelo
model = crear_modelo(TAM_IMG, TAM_IMG, activ='leaky_relu')
model.compile(optimizer=optimizers.Adam(learning_rate=0.0005),
                       loss='mean_absolute_error')

model.summary()

In [ ]:
histories.append(entrenar_modelo(model, dataset_entrena, dataset_valida))

In [ ]:
ini_imgs = 20
num_imgs = 5

# Obtener un batch del dataset de validación
test_batch = next(iter(dataset_valida.take(1)))
test_gray, test_color = test_batch

# Generar predicciones
predictions = model.predict(test_gray)

# Mostrar comparaciones
plot_comparasion(test_batch, predictions, num_images=num_imgs, start_index=ini_imgs)

### Transpose2D

In [ ]:
from tensorflow.keras import layers, models, optimizers, activations

def crear_modelo_transpose(img_h, img_w, activ='relu'):
    inputs = layers.Input(shape=(img_h, img_w, 1))

    # --- Encoder ---
    x = layers.Conv2D(64, (3,3), activation=activ, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, (3,3), activation=activ, padding='same', strides=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(128, (3,3), activation=activ, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(128, (3,3), activation=activ, padding='same', strides=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(256, (3,3), activation=activ, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(256, (3,3), activation=activ, padding='same', strides=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(512, (3,3), activation=activ, padding='same')(x)
    x = layers.BatchNormalization()(x)
    #x = layers.Conv2D(512, (3,3), activation=activ, padding='same')(x)
    #x = layers.BatchNormalization()(x)

    # --- Decoder ---
    x = layers.Conv2DTranspose(256, (3,3), activation=activ, padding='same', strides=2)(x)
    x = layers.Conv2D(256, (3,3), activation=activ, padding='same')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2DTranspose(128, (3,3), activation=activ, padding='same', strides=2)(x)
    x = layers.Conv2D(128, (3,3), activation=activ, padding='same')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2DTranspose(64, (3,3), activation=activ, padding='same', strides=2)(x)
    x = layers.Conv2D(64, (3,3), activation=activ, padding='same')(x)
    x = layers.BatchNormalization()(x)

    outputs = layers.Conv2D(3, (3,3), activation='sigmoid', padding='same')(x)

    return models.Model(inputs=inputs, outputs=outputs)

    
# Crear y compilar el modelo
model = crear_modelo_transpose(TAM_IMG, TAM_IMG, activ='leaky_relu')
model.compile(optimizer=optimizers.Adam(learning_rate=0.0005),
                       loss='mean_absolute_error')

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 96, 96, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 96, 96, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 96, 96, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 48, 48, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 48, 48, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 48, 48, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 48, 48, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 24, 24, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 24, 24, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 24, 24, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 12, 12, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 12, 12, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 12, 12, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 12, 12, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 24, 24, 256)    │     1,179,904 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 24, 24, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 24, 24, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 48, 48, 128)    │       295,040 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 48, 48, 128)    │       147,58

 Total params: 4,656,899 (17.76 MB)

 Trainable params: 4,653,187 (17.75 MB)

 Non-trainable params: 3,712 (14.50 KB)

**Entrenamiento**

In [ ]:
histories.append(entrenar_modelo(model, dataset_entrena, dataset_valida))

In [ ]:
ini_imgs = 20
num_imgs = 5

# Obtener un batch del dataset de validación
test_batch = next(iter(dataset_valida.take(1)))
test_gray, test_color = test_batch

# Generar predicciones
predictions = model.predict(test_gray)

# Mostrar comparaciones
plot_comparasion(test_batch, predictions, num_images=num_imgs, start_index=ini_imgs)

### Reescale (pregunta)

In [ ]:
from tensorflow.keras import layers, models, optimizers, activations

def crear_modelo(img_h, img_w, activ='relu'):
    inputs = layers.Input(shape=(img_h, img_w, 1))

    # --- Encoder ---
    x = layers.Conv2D(64, (3,3), activation=activ, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, (3,3), activation=activ, padding='same', strides=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(128, (3,3), activation=activ, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(128, (3,3), activation=activ, padding='same', strides=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(256, (3,3), activation=activ, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(256, (3,3), activation=activ, padding='same', strides=2)(x)
    x = layers.BatchNormalization()(x)

    # --- Bottleneck ---
    x = layers.Conv2D(512, (3,3), activation=activ, padding='same')(x)
    x = layers.BatchNormalization()(x)
    #x = layers.Conv2D(512, (3,3), activation=activ, padding='same')(x)
    #x = layers.BatchNormalization()(x)

    # --- Decoder ---
    # -------------------------------- ¿CUÁL ES LA CAPA REESCALE? --------------------------
    x = layers.UpSampling2D(size=(2, 2))(x)
    x = layers.Reescale(256, (3,3), activation=activ, padding='same')(x)
    x = layers.BatchNormalization()(x)

    x = layers.UpSampling2D(size=(2, 2))(x)
    x = layers.Conv2D(128, (3,3), activation=activ, padding='same')(x)
    x = layers.BatchNormalization()(x)

    x = layers.UpSampling2D(size=(2, 2))(x)
    x = layers.Conv2D(64, (3,3), activation=activ, padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Output
    outputs = layers.Conv2D(3, (3,3), activation='sigmoid', padding='same')(x)

    return models.Model(inputs=inputs, outputs=outputs)

    
# Crear y compilar el modelo
model = crear_modelo(TAM_IMG, TAM_IMG, activ='leaky_relu')
model.compile(optimizer=optimizers.Adam(learning_rate=0.0005),
                       loss='mean_absolute_error')

model.summary()

In [ ]:
histories.append(entrenar_modelo(model, dataset_entrena, dataset_valida))

In [ ]:
ini_imgs = 20
num_imgs = 5

# Obtener un batch del dataset de validación
test_batch = next(iter(dataset_valida.take(1)))
test_gray, test_color = test_batch

# Generar predicciones
predictions = model.predict(test_gray)

# Mostrar comparaciones
plot_comparasion(test_batch, predictions, num_images=num_imgs, start_index=ini_imgs)

### Comparaciones

In [4]:
def visualizar_historiales(histories):
    """
    Crea gráficos de 'loss' y 'val_loss' para una lista de historiales de entrenamiento.
    Cada historial se muestra en un subgráfico separado, dispuesto horizontalmente.

    Args:
        histories (list): Una lista de objetos History de Keras (o diccionarios simulados).
                          Se esperan 2 o 3 elementos en la lista.
    """
    num_plots = len(histories)
    if num_plots == 0:
        print("La lista de historiales está vacía.")
        return

    # Crear una figura con subgráficos: 1 fila, y una columna por cada historial
    fig = make_subplots(
        rows=1,
        cols=num_plots,
        subplot_titles=[f"Entrenamiento {i+1}" for i in range(num_plots)]
    )

    # Iterar sobre cada historial en la lista para agregarlo a su subgráfico
    for i, history in enumerate(histories):
        # El historial puede ser un objeto de Keras o un diccionario
        history_dict = history.history if hasattr(history, 'history') else history
        
        # Validar que las métricas necesarias existan
        if 'loss' not in history_dict or 'val_loss' not in history_dict:
            print(f"El historial {i+1} no contiene 'loss' y/o 'val_loss'.")
            continue

        epochs = list(range(1, len(history_dict['loss']) + 1))

        # Añadir la curva de 'loss' (pérdida de entrenamiento)
        fig.add_trace(go.Scatter(
            x=epochs,
            y=history_dict['loss'],
            mode='lines+markers',
            name='Loss',
            legendgroup=f'group{i}', # Agrupa leyendas para este subgráfico
            line=dict(color='royalblue'),
            marker=dict(size=4)
        ), row=1, col=i + 1) # Especifica la posición del subgráfico

        # Añadir la curva de 'val_loss' (pérdida de validación)
        fig.add_trace(go.Scatter(
            x=epochs,
            y=history_dict['val_loss'],
            mode='lines+markers',
            name='Val-Loss',
            legendgroup=f'group{i}',
            line=dict(color='crimson'),
            marker=dict(size=4)
        ), row=1, col=i + 1)

    # Actualizar el diseño general de la figura
    fig.update_layout(
        title_text='Comparación de Resultados de Entrenamiento',
        height=400,  # Altura reducida para mejor visualización
        width=400 * num_plots, # Ancho dinámico basado en la cantidad de gráficos
        template='plotly_white',
        showlegend=True
    )
    
    # Actualizar títulos de los ejes para todos los subgráficos
    fig.update_xaxes(title_text='Época')
    fig.update_yaxes(title_text='Pérdida (Loss)')

    return fig

fig = visualizar_historiales(histories)
# fig.show()

NameError: name 'histories' is not defined